In [1]:
# search in the hyperparameter space with W&B sweep
import logging
from ast import literal_eval as load
import pandas as pd
import numpy as np
from ast import literal_eval
import sklearn

from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)


In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df = pd.read_csv('triples.csv')
df = df.rename(columns={'idx': 'indx'})
df.insert(loc=0, column='idx', value=np.arange(len(df)))
sent_num=len(df)

In [3]:
def convert(arr):
    trip_list=[] # not trip_ls
    ls=[]
    for i in range(len(arr)):
        uni_name=arr[i,1]
        uni_name=(uni_name[0].upper()+uni_name[1:]).replace('-',' ')
        if arr[i,3]!='[]' and arr[i,4]!='[]':
            pre = load(arr[i,3])[0]
            np = load(arr[i,4])[0]
            if pre[1][0]<np[1][0]:
                triple=[uni_name,pre[0],np[0]]
                trip_list.append(triple)
                
                word_ls = arr[i,2].split(' ')
                word_ls.insert(pre[1][0], '<<')
                word_ls.insert(pre[1][1]+1, '>>')
                word_ls.insert(np[1][0]+2, '[[')
                word_ls.insert(np[1][1]+3, ']]')
                unit = arr[i,1]
                unit = (unit[0].upper()+unit[1:]).replace('-',' ')
                unit_ls = ['[[']+(unit.split(' '))+[']]']
                word_ls = unit_ls+[':']+word_ls
                flg=0
                if arr[i,7]=='[]':
                    trip_ls = []
                else:
                    trip_ls = load(arr[i,7])
                    for trip in trip_ls:
                        if trip[1]==pre[0] and trip[2]==np[0]:
                            flg=1
                            break
                ls.append([int(arr[i, 0]), unit, pre[0], np[0], trip_ls, ' '.join(word_ls), flg])
    dataframe = pd.DataFrame(ls)
    dataframe.columns = ['idx', 'info_unit', 'pre', 'np', 'triples', 'text', 'labels']
    return dataframe,trip_list

In [4]:
df=df.values
df,trip_list=convert(df)

In [5]:
model_args = ClassificationArgs()

model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.manual_seed = 1
model_args.fp16 = False
model_args.use_multiprocessing = True
model_args.do_lower_case = True  # when using uncased model

In [6]:
def triple_F1(ref, pred):
    return 0

In [7]:
# Create a TransformerModel
model = ClassificationModel(
    "bert",
    "../rel/outputsC/best_model",
    args=model_args,
)
result, model_outputs, wrong_predictions = model.eval_model(df, F1_score=triple_F1)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/1483 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/186 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 0, 'tn': 282, 'fp': 1201, 'fn': 0, 'F1_score': 0, 'eval_loss': 2.5842002955295387}


In [8]:
preds = list(model_outputs.argmax(axis=1))
df['preds']=preds
df['cand']=trip_list
df.loc[df['preds']==0,'cand']=None
data=[]
for i in range(sent_num):
    temp = list(df[df['idx']==i]['cand'])
    temp = [t for t in temp if t]
    data.append(str(temp))

In [9]:
data

['[]',
 "[['Model', 'introduce', 'recurrent neural network grammars ( RNNGs']]",
 "[['Model', 'give', 'two variants']]",
 '[]',
 "[['Model', 'present', 'simple importance sampling algorithm']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Approach', 'show', 'even larger performance gains']]",
 "[['Approach', 'introducing', 'cloze - style training objective']]",
 "[['Experimental setup', 'subsample', 'up to 18B tokens']]",
 '[]',
 '[]',
 "[['Experimental setup', 'on', 'DGX - 1 machines']]",
 "[['Experimental setup', 'use', 'NCCL2 library']]",
 "[['Experimental setup', 'train', 'models']]",
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Research problem', 'investigates', 'general purpose sequence - to - sequence models']]",
 "[['Research problem', 'incorporate', 'sequenceto - sequence model']]",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Hyperparameters', 'For', 'data pre-processing']]",
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Results', 'using', 'subword information']]",
 '[]',
 '[]',
 '[]',
 '[]',
 "[['Resu

In [10]:
len(data)

3149

In [12]:
data=pd.DataFrame(data,columns=['triple_C'])
data

,triple_C
0,[]
1,"[['Model', 'introduce', 'recurrent neural netw..."
2,"[['Model', 'give', 'two variants']]"
3,[]
4,"[['Model', 'present', 'simple importance sampl..."
...,...
3144,[]
3145,"[['Approach', 'as', 'BERT joint']]"
3146,"[['Hyperparameters', 'initialized', 'our model']]"
3147,"[['Hyperparameters', 'trained', 'model']]"


In [13]:
data.to_csv('C.csv',index=False)